In [10]:
import tensorflow as tf
from tensorflow import keras
from keras.saving import load_model
from keras.utils import register_keras_serializable

print(keras.__version__)

# Need the below function to load the model
@register_keras_serializable()
def ssim_loss(y_true, y_pred):
  """
  Compute the SSIM loss between the true and predicted spectrograms.
  Parameters:
  y_true (tf.Tensor): True spectrogram.
  y_pred (tf.Tensor): Predicted spectrogram.
  """
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))


loaded_model = load_model('modeldev_luke.keras')

loaded_model.summary()

3.5.0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 32, 1)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 32, 64)    │            640 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 128, 32, 64)    │         36,928 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 64, 16, 64)     │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 64, 16, 128)    │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 64, 16, 128)    │        147,584 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 32, 8, 128)     │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 32, 8, 256)     │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d             │ (None, 64, 16, 256)    │              0 │ conv2d_4[0][0]         │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 64, 16, 384)    │              0 │ up_sampling2d[0][0],   │
│                           │                        │                │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 16, 128)    │        442,496 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 64, 16, 128)    │        147,584 │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_1           │ (None, 128, 32, 128)   │              0 │ conv2d_6[0][0]         │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 128, 32, 192)   │              0 │ up_sampling2d_1[0][0], │
│ (Concatenate)             │                        │                │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 128, 32, 64)    │        110,656 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 128, 32, 64)    │         36,928 │ conv2d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)    

 Total params: 2,583,812 (9.86 MB)

 Trainable params: 1,291,905 (4.93 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,291,907 (4.93 MB)

In [12]:
"""
Created on Wed Nov 27 11:55:53 2024
@author: Ayush
"""

import os
import random
import librosa
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import librosa.display

# Preprocessing class for handling audio files
class Preprocessing:
    def load_audio(self, file_path: str) -> np.ndarray:
        """
        Load an audio file and return its waveform as a numpy array.
        Parameters:
        file_path (str): Path to the audio file.
        Returns:
        np.ndarray: Audio waveform, and sample rate.
        """
        audio, sr = librosa.load(file_path, sr=None)  # Load audio with original sampling rate
        return audio, sr

    def extract_spectrogram(self, audio: np.ndarray, sr: int = 22050) -> np.ndarray:
        """
        Convert the audio waveform to a Mel spectrogram.
        Parameters:
        audio (np.ndarray): The audio waveform.
        sr (int): The sampling rate of the audio.
        Returns:
        np.ndarray: Spectrogram (magnitude in decibels).
        """
        spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=2048, hop_length=512)
        spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
        return spectrogram_db

    def save_spectrogram(self, spectrogram: np.ndarray, file_path: str):
        """
        Save the spectrogram as a .npy file.
        Parameters:
        spectrogram (np.ndarray): The spectrogram to save.
        file_path (str): Path where to save the spectrogram file.
        """
        np.save(file_path, spectrogram)

    def match_length(self, audio: np.ndarray, target_length: int) -> np.ndarray:
        """
        Match the length of the audio to a target length by trimming or padding.
        Parameters:
        audio (np.ndarray): Input audio waveform.
        target_length (int): Target length for the audio.
        Returns:
        np.ndarray: Length-matched audio waveform.
        """
        if len(audio) < target_length:
            # Pad with zeros if shorter
            audio = np.pad(audio, (0, target_length - len(audio)), mode='constant')
        else:
            # Trim if longer
            audio = audio[:target_length]
        return audio

    def match_length_random(self, audio: np.ndarray, target_length: int) -> np.ndarray:
        """
        Match the length of the audio to a target length by choosing a random start point.
        Parameters:
        audio (np.ndarray): Input audio waveform.
        target_length (int): Target length for the audio.
        Returns:
        np.ndarray: Length-matched audio waveform.
        """
        if len(audio) < target_length:
            # Pad with zeros if shorter
            audio = np.pad(audio, (0, target_length - len(audio)), mode='constant')
        else:
            # select a random segment from the audio
            start = random.randint(0, len(audio) - target_length)
            audio = audio[start:start + target_length]
        return audio

# Function to get all noise files from a directory
def get_noise_files(noise_dir: str) -> list:
    """
    Retrieve all noise file paths from the specified directory.
    Parameters:
    noise_dir (str): Path to the directory containing noise files.
    Returns:
    list: List of noise file paths.
    """
    noise_files = []
    for root, _, files in os.walk(noise_dir):
        for file in files:
            if file.endswith(('.wav', '.flac')):  # Process only supported audio files
                noise_files.append(os.path.join(root, file))
    return noise_files


In [13]:
# Need the below function to extract spectrograms from audio in the form of an
# np.ndarray
def extract_windows(audio: np.ndarray, preprocessing: Preprocessing, sample_rate: int = 16000, window_duration: float = 1.02):
  """
  Extract windows of sample_rate * window_duration from the audio file. Extract
  spectrograms from them and return a list with all the spectrograms.
  Parameters:
  audio (np.ndarray): audio file to extract windows from.
  preprocessing (Preprocessing): Instance of preprocessing class.
  sample_rate (int): sampling rate of the audio file.
  window_duration (float): duration of the windows in seconds.
  """
  window_samples = int(sample_rate * window_duration)

  spectrograms = []

  for start in range(0, len(audio), window_samples):
      end = start + window_samples

      window = audio[start:end]

      if len(window) < window_samples:
        # pad with silence until the window is filled
        padding = window_samples - len(window)
        window = np.pad(window, (0, padding), mode='constant')

      spectrogram = preprocessing.extract_spectrogram(window, sample_rate)
      spectrograms.append(spectrogram)

  return spectrograms

In [14]:
# Need the below function to get the spectrogram in the right shape for training
# and to normalize the values
def add_single_channel_and_normalize(spectrograms: list[np.ndarray]) -> list[np.ndarray]:
  """
  Add single channel to the spectrogram [turn shape from (x, 128, 32) to
  (x, 128, 32, 1)] and normalize it.
  Parameters:
  spectrograms (list[np.ndarray]): List of spectrogram arrays.
  """
  min_val = -80
  max_val = 0

  rgb_spectrograms = np.repeat(np.array(spectrograms)[..., np.newaxis], 1, axis=-1)
  normalized_spectrograms = (rgb_spectrograms - min_val) / (max_val - min_val)

  return normalized_spectrograms

In [15]:
# Need the below function to inverse spectrogram into audio
def inverse_spectrogram(spectrogram: np.ndarray, volume_factor: int = 1):
  """
  Turn the spectrogram back into audio.
  Parameters:
  spectrogram (np.ndarray): The spectrogram to turn back into audio.
  volume_factor (int): The volume factor to apply to the audio.
  """
  mel_spectrogram = librosa.db_to_power(spectrogram)
  linear_spectrogram = librosa.feature.inverse.mel_to_stft(mel_spectrogram, sr=16000, n_fft=2048)
  # Feed free to experiment with n_iter -> larger values I think are more accurate,
  # but also take longer
  audio_signal = librosa.griffinlim(linear_spectrogram, n_iter=64, hop_length=512, n_fft=2048)
  audio_signal *= volume_factor
  return audio_signal

In [16]:
# Inverse spectrogram into audio using phase information
def inverse_spectrogram_with_phase(spectrogram: np.ndarray, phase, volume_factor: int = 1, ):
  """
  Turn the spectrogram back into audio.
  Parameters:
  spectrogram (np.ndarray): The spectrogram to turn back into audio.
  volume_factor (int): The volume factor to apply to the audio.
  """
  mel_spectrogram = librosa.db_to_power(spectrogram)
  linear_spectrogram = librosa.feature.inverse.mel_to_stft(mel_spectrogram, sr=16000, n_fft=2048)

  min_time_frames = min(linear_spectrogram.shape[1], phase.shape[1])
  linear_spectrogram = linear_spectrogram[:, :min_time_frames]
  phase = phase[:, :min_time_frames]

  reconstructed_stft = linear_spectrogram * np.exp(1j * phase)

  audio_signal = librosa.istft(reconstructed_stft, hop_length=512, n_fft=2048)
  audio_signal *= volume_factor
  return audio_signal

In [25]:
# Denoise audio from start to finish using no phase information
def denoise_audio(audio_file: str, model: keras.Model):
  """
  Denoise the audio file using the given model.
  Parameters:
  audio_file (str): The path to the audio file to denoise.
  model (keras.Model): The model to use for denoising.
  """
  audio, sr = librosa.load(audio_file, sr=16000)

  # Preprocess the audio file, turning it into spectrograms
  preprocessing = Preprocessing()
  extracted_spectrograms = extract_windows(audio, preprocessing, 16000, 1.02)
  normalized_spectrograms = add_single_channel_and_normalize(extracted_spectrograms)

  # Get predictions
  predictions = model.predict(normalized_spectrograms)

  min_val = -80
  max_val = 0

  # Process the predicted spetrograms back into audio
  unscaled_predictions = predictions * (max_val - min_val) + min_val
  predicted_spectrograms = np.squeeze(unscaled_predictions, axis=-1)
  combined_spectrogram = np.hstack(predicted_spectrograms)
  predicted_audio = inverse_spectrogram(combined_spectrogram)

  return predicted_audio

In [18]:
# Denoise audio from start to finish using phase information
def denoise_audio_with_phase(audio_file: str, model: keras.Model):
  """
  Denoise the audio file using the given model.
  Parameters:
  audio_file (str): The path to the audio file to denoise.
  model (keras.Model): The model to use for denoising.
  """
  audio, sr = librosa.load(audio_file, sr=16000)

  stft_result = librosa.stft(audio, n_fft=2048, hop_length=512)
  phase = np.angle(stft_result)
  magnitude = np.abs(stft_result)

  # Preprocess the audio file, turning it into spectrograms
  preprocessing = Preprocessing()
  extracted_spectrograms = extract_windows(audio, preprocessing, 16000, 1.02)
  normalized_spectrograms = add_single_channel_and_normalize(extracted_spectrograms)

  # Get predictions
  predictions = model.predict(normalized_spectrograms)

  min_val = -80
  max_val = 0

  # Process the predicted spetrograms back into audio
  unscaled_predictions = predictions * (max_val - min_val) + min_val
  predicted_spectrograms = np.squeeze(unscaled_predictions, axis=-1)
  combined_spectrogram = np.hstack(predicted_spectrograms)

  predicted_audio = inverse_spectrogram_with_phase(combined_spectrogram, phase)

  return predicted_audio

In [21]:
from IPython.display import Audio

# Original Audio
Audio('noisy_1_8555-292519-0009.flac')

In [22]:
# Example of denoising using phase info
denoised_audio_with_phase = denoise_audio_with_phase('noisy_1_8555-292519-0009.flac', loaded_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [23]:
display(Audio(denoised_audio_with_phase, rate=16000))

In [27]:
# Example of denoising without phase info
denoised_audio = denoise_audio('noisy_1_8555-292519-0009.flac', loaded_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [28]:
display(Audio(denoised_audio, rate=16000))

In [44]:
# Denoising without needing any of the above functions
def standalone_denoise_audio_with_phase(audio_file: str, model: keras.Model):
  """
  Denoise the audio file using the given model.
  Parameters:
  audio_file (str): The path to the audio file to denoise.
  model (keras.Model): The model to use for denoising.
  """
  audio, sample_rate = librosa.load(audio_file, sr=16000)

  stft_result = librosa.stft(audio, n_fft=2048, hop_length=512)
  phase = np.angle(stft_result)
  magnitude = np.abs(stft_result)

  window_duration = 1.02
  window_samples = int(sample_rate * window_duration)
  spectrograms = []

  # extract spectrograms from chunks of the audio
  for start in range(0, len(audio), window_samples):
      end = start + window_samples

      window = audio[start:end]

      if len(window) < window_samples:
        # pad with silence until the window is filled
        padding = window_samples - len(window)
        window = np.pad(window, (0, padding), mode='constant')

      # spectrogram = preprocessing.extract_spectrogram(window, sample_rate)
      spec = librosa.feature.melspectrogram(y=window, sr=sample_rate, n_fft=2048, hop_length=512)
      spec_db = librosa.power_to_db(spec, ref=np.max)
      spectrograms.append(spec_db)

  min_val = -80
  max_val = 0

  rgb_spectrograms = np.expand_dims(spectrograms, axis=-1)
  normalized_spectrograms = (rgb_spectrograms - min_val) / (max_val - min_val)

  # Get predictions
  predictions = model.predict(normalized_spectrograms)

  min_val = -80
  max_val = 0

  # Process the predicted spetrograms back into audio
  unscaled_predictions = predictions * (max_val - min_val) + min_val
  predicted_spectrograms = np.squeeze(unscaled_predictions, axis=-1)
  combined_spectrogram = np.hstack(predicted_spectrograms)

  mel_spectrogram = librosa.db_to_power(combined_spectrogram)
  linear_spectrogram = librosa.feature.inverse.mel_to_stft(mel_spectrogram, sr=16000, n_fft=2048)

  min_time_frames = min(linear_spectrogram.shape[1], phase.shape[1])
  linear_spectrogram = linear_spectrogram[:, :min_time_frames]
  phase = phase[:, :min_time_frames]

  reconstructed_stft = linear_spectrogram * np.exp(1j * phase)

  audio_signal = librosa.istft(reconstructed_stft, hop_length=512, n_fft=2048)

  return audio_signal

In [42]:
standalone_denoised = standalone_denoise_audio_with_phase('noisy_1_8555-292519-0009.flac', loaded_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [43]:
display(Audio(standalone_denoised, rate=16000))